In [1]:
from EnsembleFramework import Framework

In [5]:
from torch_geometric.datasets import PPI
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from torch_geometric.loader import DataLoader

train_dataset = PPI(root='/tmp/PPI', split="train")
val_dataset = PPI(root='/tmp/PPI', split="val")
test_dataset = PPI(root='/tmp/PPI', split="test")
train_dataset.transform = T.NormalizeFeatures()
val_dataset.transform = T.NormalizeFeatures()
test_dataset.transform = T.NormalizeFeatures()

train_loader = iter(DataLoader(train_dataset, batch_size=len(train_dataset)))
val_loader = iter(DataLoader(val_dataset, batch_size=len(val_dataset)))
test_loader = iter(DataLoader(test_dataset, batch_size=len(test_dataset)))

In [6]:
train_set = next(train_loader)
test_set = next(test_loader)
val_set = next(val_loader)

In [7]:
from EnsembleFramework import Framework
import torch
from torch import nn

def user_function(kwargs):
    return  nn.functional.normalize(kwargs["updated_features"] + kwargs["summed_neighbors"], p = 2.0, dim = -1)
    
framework = Framework(hops_list= [5], clfs=[], attention_configs=[{'inter_layer_normalize': False,
   'use_pseudo_attention': True,
   'cosine_eps': 0.01,
   'dropout_attn': None}], handle_nan=0.0, gpu_idx=2, user_functions=[user_function])
new_train_features = framework.get_features(X=train_set.x, edge_index=train_set.edge_index, mask=torch.ones(train_set.x.shape[0]).type(torch.bool))
new_test_features = framework.get_features(X=test_set.x, edge_index=test_set.edge_index, mask=torch.ones(test_set.x.shape[0]).type(torch.bool))

/home/dwalke/git/graph_aware_ml/EnsembleFramework.py:145: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:230.)
  summed_exp_score = torch.zeros_like(exp_score).scatter(0, target,exp_score, reduce="add")


In [50]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
model = MultiOutputClassifier(XGBClassifier(tree_method='hist',
                      device="cuda:2", max_depth = None, eta = .725)).fit(new_train_features[0].cpu(), train_set.y)

[CV 1/5; 2/5] START eta=0.675, max_depth=None...................................
[CV 1/5; 2/5] END eta=0.675, max_depth=None;, score=(train=0.998, test=0.866) total time= 8.1min
[CV 3/5; 3/5] START eta=0.7, max_depth=None.....................................
[CV 3/5; 3/5] END eta=0.7, max_depth=None;, score=(train=0.997, test=0.869) total time= 7.6min
[CV 1/5; 5/5] START eta=0.75, max_depth=None....................................
[CV 1/5; 5/5] END eta=0.75, max_depth=None;, score=(train=0.998, test=0.864) total time= 5.8min


In [51]:
from sklearn.metrics import f1_score
f1_score(test_set.y, model.predict(new_test_features[0].cpu()), average ="micro")

[CV 5/5; 1/5] START eta=0.65, max_depth=None....................................
[CV 5/5; 1/5] END eta=0.65, max_depth=None;, score=(train=0.996, test=0.946) total time= 9.0min
[CV 1/5; 4/5] START eta=0.725, max_depth=None...................................
[CV 1/5; 4/5] END eta=0.725, max_depth=None;, score=(train=0.998, test=0.866) total time= 7.4min
[CV 2/5; 5/5] START eta=0.75, max_depth=None....................................
[CV 2/5; 5/5] END eta=0.75, max_depth=None;, score=(train=0.997, test=0.862) total time= 5.4min
[CV 2/5; 2/5] START eta=0.675, max_depth=None...................................
[CV 2/5; 2/5] END eta=0.675, max_depth=None;, score=(train=0.997, test=0.864) total time= 8.1min
[CV 4/5; 3/5] START eta=0.7, max_depth=None.....................................
[CV 4/5; 3/5] END eta=0.7, max_depth=None;, score=(train=0.997, test=0.931) total time= 9.1min
[CV 4/5; 5/5] START eta=0.75, max_depth=None....................................
[CV 4/5; 5/5] END eta=0.75, max_d

0.93574142449241

In [52]:
import numpy as np
np.equal(model.predict(new_test_features[0].cpu()), test_set.y).sum()/ test_set.y.numel()

[CV 1/5; 1/5] START eta=0.65, max_depth=None....................................
[CV 1/5; 1/5] END eta=0.65, max_depth=None;, score=(train=0.998, test=0.866) total time= 8.1min
[CV 2/5; 3/5] START eta=0.7, max_depth=None.....................................
[CV 2/5; 3/5] END eta=0.7, max_depth=None;, score=(train=0.997, test=0.864) total time= 9.0min
[CV 3/5; 5/5] START eta=0.75, max_depth=None....................................
[CV 3/5; 5/5] END eta=0.75, max_depth=None;, score=(train=0.997, test=0.869) total time= 4.8min
[CV 4/5; 2/5] START eta=0.675, max_depth=None...................................
[CV 4/5; 2/5] END eta=0.675, max_depth=None;, score=(train=0.996, test=0.930) total time= 8.2min
[CV 5/5; 3/5] START eta=0.7, max_depth=None.....................................
[CV 5/5; 3/5] END eta=0.7, max_depth=None;, score=(train=0.997, test=0.947) total time= 9.2min
[CV 5/5; 5/5] START eta=0.75, max_depth=None....................................
[CV 5/5; 5/5] END eta=0.75, max_dep

tensor(0.9623)

In [47]:
from sklearn.model_selection import GridSearchCV

def get_best_estimator(model, param_grid):
    hops_list = [5]
    framework = Framework([user_function for _ in hops_list], 
                         hops_list=hops_list, ## to obtain best for local neighborhood
                         clfs=[],
                         gpu_idx=0,
                         handle_nan=0.0,
                        attention_configs=[{'inter_layer_normalize': True,
                             'use_pseudo_attention':True,
                             'cosine_eps':.01,
                             'dropout_attn': None}])
    X_train = framework.get_features(train_set.x, train_set.edge_index, torch.ones(train_set.x.shape[0]).type(torch.bool))
    X_train = X_train[0].cpu()
    
    grid = GridSearchCV(
    n_jobs=10,
    cv=5,
    estimator=model,
    scoring="f1_micro",
    refit=True,
    return_train_score=True,
    param_grid=param_grid,
    verbose= 10
    )
    grid.fit(X_train, train_set.y)
    return grid.best_estimator_
    

In [55]:
from sklearn.linear_model import LogisticRegression

param_grid = {
    "max_depth": [None],
    "eta": [.725],
    "gamma": [0],
    "lambda": [0, 1, 5, 10],
}
xgBoost = get_best_estimator(XGBClassifier(tree_method='hist',
                      device="cuda:2"), param_grid)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [61]:
xgBoost.get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': 'cuda:2',
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': 0,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': 'hist',
 'validate_parameters': None,
 'verbosity': None,
 'eta': 0.725,
 'lambda': 1}

[CV 2/5; 2/4] START eta=0.725, gamma=0, lambda=1, max_depth=None................
[CV 2/5; 2/4] END eta=0.725, gamma=0, lambda=1, max_depth=None;, score=(train=0.997, test=0.863) total time= 8.7min
[CV 5/5; 3/4] START eta=0.725, gamma=0, lambda=5, max_depth=None................
[CV 5/5; 3/4] END eta=0.725, gamma=0, lambda=5, max_depth=None;, score=(train=0.996, test=0.945) total time= 8.2min
[CV 5/5; 1/4] START eta=0.725, gamma=0, lambda=0, max_depth=None................
[CV 5/5; 1/4] END eta=0.725, gamma=0, lambda=0, max_depth=None;, score=(train=0.997, test=0.947) total time= 7.8min
[CV 1/5; 3/4] START eta=0.725, gamma=0, lambda=5, max_depth=None................
[CV 1/5; 3/4] END eta=0.725, gamma=0, lambda=5, max_depth=None;, score=(train=0.998, test=0.862) total time= 9.3min
[CV 1/5; 1/4] START eta=0.725, gamma=0, lambda=0, max_depth=None................
[CV 1/5; 1/4] END eta=0.725, gamma=0, lambda=0, max_depth=None;, score=(train=0.998, test=0.864) total time= 8.5min
[CV 2/5; 3/4] S

In [7]:
import xgboost as xgb
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from hyperopt import fmin, tpe, hp,STATUS_OK, SparkTrials
from sklearn.metrics import accuracy_score, f1_score
from tqdm.notebook import tqdm
from sklearn.multioutput import MultiOutputClassifier

booster_self = ["gbtree"]
n_estimators_self = [5, 50, 100,150, 200, 400, 500, 800, 1_000, 1_200]
max_depth_self = [None, 1, 2, 3, 4]
max_delta_step_self = [None, 1, 2, 3, 4]
min_child_weight_self = [None, 1, 2, 3, 4]

xb_choices = {
    'booster': booster_self,
    'n_estimators': n_estimators_self,
    'max_depth': max_depth_self,
    'max_delta_step': max_delta_step_self,
    'min_child_weight': min_child_weight_self,
}
space_xb = {
    **{key: hp.choice(key, value) for key, value in xb_choices.items()},
    'eta': hp.loguniform('eta', -3, -.3),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'reg_lambda': hp.uniform('reg_lambda', 0, 1),
    'alpha': hp.uniform('alpha', 0, 1),
}


def search_hyperparams():    
    stores=[]
    hops_lists = [[0,0, 0], [3,3, 3], [5,5, 5]]
    for hops_list in hops_lists:
        store=dict({})
        clfs = [XGBClassifier]
        spaces= [space_xb, space_svc, space_lr]
        space_choices= [xb_choices, svc_choices, {}]
        hops_list = hops_list
        attention_configs = [ None, 
                             {'inter_layer_normalize': False,
                             'use_pseudo_attention':True,
                             'cosine_eps':.01,
                             'dropout_attn': None},
                            {'inter_layer_normalize': True,
                             'use_pseudo_attention':True,
                             'cosine_eps':.01,
                             'dropout_attn': None}]
        max_acc = 0
        best_hops = None
        best_user_function = None
        best_attention_config = None
        best_clf_params = None
        best_clf = None
        for clf_idx, clf in tqdm(enumerate(clfs)):
            space = spaces[clf_idx]
            framework = Framework([user_function for _ in hops_list], 
                         hops_list=hops_list, ## to obtain best for local neighborhood
                         clfs=[],
                         gpu_idx=0,
                         handle_nan=0.0,
                        attention_configs=attention_configs)
    
    
            val_aggregated_feature_list = framework.get_features(val_set.x, val_set.edge_index, torch.ones(val_set.x.shape[0]).type(torch.bool))
            train_aggregated_feature_list = framework.get_features(train_set.x, train_set.edge_index, torch.ones(train_set.x.shape[0]).type(torch.bool))## this shoould be indepent of the length of the classifiers list
    
            for i in range(len(val_aggregated_feature_list)):
                print(clf)
                print(hops_list[i])
                X_val = val_aggregated_feature_list[i].cpu()
                X_train = train_aggregated_feature_list[i].cpu()
                def objective(params):
                    model = MultiOutputClassifier(clf(**params, random_state = 42,tree_method='hist',
                      device="cuda"), n_jobs=1).fit(X_train, train_set.y)
                    y_pred = model.predict(X_val)
                    score = f1_score(val_set.y, y_pred, average ="micro")
                    return {'loss': -score, 'status': STATUS_OK}
                spark_trials = SparkTrials()
                best_params = fmin(objective, space, algo=tpe.suggest, max_evals=500, trials=spark_trials)
    
                for key,value in space_choices[clf_idx].items():
                    best_params[key] = value[best_params[key]]
    
                best_model = MultiOutputClassifier(clf(**best_params, random_state = 42,tree_method='hist',
                      device="cuda"), n_jobs=1).fit(X_train, train_set.y)
                best_model.fit(X_train, train_set.y)
                y_pred = best_model.predict(X_val)
                score = f1_score(val_set.y, y_pred, average ="micro")
                if score >= max_acc:
                    max_acc = score
                    best_hops = hops_list[i]
                    best_user_function = user_functions[i]
                    best_attention_config = attention_configs[i]
                    best_clf_params = best_params
                    best_clf = clf
        store["hops_list"] = hops_list
        store["max_acc"] = max_acc
        store["best_hops"] = best_hops
        store["best_user_function"] = best_user_function
        store["best_attention_config"] = best_attention_config
        store["best_clf_params"] = best_clf_params
        store["best_clf"] = best_clf
        stores.append(store)
    return stores

In [ ]:
ppi_stores = search_hyperparams()

0it [00:00, ?it/s]

Because the requested parallelism was None or a non-positive value, parallelism will be set to (64), which is Spark's default parallelism (64), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


<class 'xgboost.sklearn.XGBClassifier'>
0

  0%|                                                                           | 0/500 [00:00<?, ?trial/s, best loss=?]


  0%|                                           | 1/500 [01:49<15:09:42, 109.38s/trial, best loss: -0.39624524210302836]


  0%|▏                                            | 2/500 [02:48<11:02:18, 79.80s/trial, best loss: -0.4168114483595519]

...


 68%|███████████████████████████             | 339/500 [16:21:31<9:58:46, 223.15s/trial, best loss: -0.5416852871207114]


 68%|██████████████████████████▌            | 340/500 [16:42:53<24:02:37, 540.98s/trial, best loss: -0.5416852871207114]


 68%|██████████████████████████▌            | 341/500 [16:45:07<18:29:30, 418.68s/trial, best loss: -0.5416852871207114]


 68%|██████████████████████████▋            | 342/500 [16:49:34<16:23:09, 373.35s/trial, best loss: -0.5416852871207114]


 69%|██████████████████████████▊            | 343/500 [16:51:54<13:14:00, 303.44s/trial, best loss: -0.5416852871207114]